In [ ]:
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np
from scipy import stats, special

def rician_pdf(x, var, loc_val):
    
    x = x[np.newaxis, :]
    loc_val = loc_val[:, np.newaxis]
    
    return x * np.exp( -1 * (x ** 2 + loc_val ** 2) / var ) * special.iv(0, x * 2 * loc_val / var)

def cdf_from_pdf(x, pdf):
    
    cdf = np.zeros((pdf.shape[0], x.size))
    for i in range(x.size):
        
        cdf[:, i] = np.trapz(pdf[:, 0:i+1], x[0:i+1], axis=-1)
        
    return cdf
data = h5py.File(Path.home()/'group'/'project'/'datasets'/'data'/'220922_dl_grid_data_85to90deg_18575to18580ev_5mm.h5', 'r')
print(list(data.keys()))

In [ ]:
metadata = {
    'pitch': data['meta']['theta_min'][:],
    'energy': data['meta']['energy'][:],
}

metadata = pd.DataFrame(metadata)

#print(metadata['energy'].unique)

pitch_list = [86.]#np.linspace(85.0, 90.0, 6)
energy = 18575
inds = []

for pitch in pitch_list:
    index = metadata[(metadata['pitch']==pitch)&(metadata['energy']==energy)].index.array[0]
    inds.append(index)


In [ ]:
t = np.arange(0, 8192, 1) / 205e6
clist = sns.color_palette('deep', n_colors=10)
sns.set_theme(style='whitegrid', context='talk', font_scale=1.25)
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(24, 18), sharex=True)

for i, ind in enumerate(np.flip(inds)):

    
    axs[i%3][i//3].plot(t, data['x'][ind, 0:8192].real / 60, color=clist[i])
    axs[i%3][i//3].set_ylabel('Voltage (V)')
    axs[i%3][i//3].set_title(f'Pitch Angle = {90-i}')
    
axs[2][0].set_xlim(t[0], t[200])
axs[2][0].set_xlabel('time (s)')
axs[2][1].set_xlabel('time (s)')
    
plt.savefig(Path.home()/'group'/'project'/'plots'/'220928_example_signals', dpi=128)

In [ ]:
t = np.arange(0, 8192, 1) / 205e6
clist = sns.color_palette('deep', n_colors=10)
sns.set_theme(style='whitegrid', context='talk', font_scale=1.25)
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(24, 18), sharex=True)

for i, ind in enumerate(np.flip(inds)):

    
    axs[i%3][i//3].plot(1e-6*np.fft.fftfreq(8192, 1/205e6), (abs(np.fft.fft(data['x'][ind, 0:8192], norm='forward')/60)**2)/(50*1), color=clist[i])
    axs[i%3][i//3].set_ylabel('Power (W)')
    axs[i%3][i//3].set_title(f'Pitch Angle = {90-i}')
    
#axs[2][0].set_xlim(t[0], t[200])
axs[2][0].set_xlabel('Frequency (MHz)')
axs[2][1].set_xlabel('Frequency (MHz)')
    
plt.savefig(Path.home()/'group'/'project'/'plots'/'220928_example_signals_fft', dpi=128)

In [ ]:
t = np.arange(0, 8192, 1) / 205e6
var = 50 * 60 * 205e6 * 10 * 1.38e-23
noise = np.random.multivariate_normal([0,0], np.eye(2) * var, 8192 )
noise = noise[:, 0] + 1j * noise[:, 1]

clist = sns.color_palette('deep', n_colors=10)
sns.set_theme(style='whitegrid', context='talk', font_scale=1.25)
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(13, 8),)

#for i, ind in enumerate(np.flip(inds)):

axs.plot(1e-6*np.fft.fftfreq(8192, 1/205e6), (abs(np.fft.fft(noise, norm='forward'))**2)/50)
    
axs.plot(1e-6*np.fft.fftfreq(8192, 1/205e6), (abs(np.fft.fft(data['x'][inds[0], 0:8192], norm='forward')/1)**2)/(50*1), color=clist[1], lw=5)
axs.set_ylabel('Power (W)')
axs.set_title(f'Pitch Angle = {86}')
    
#axs[2][0].set_xlim(t[0], t[200])
axs.set_xlabel('Frequency (MHz)')

    
plt.savefig(Path.home()/'group'/'project'/'plots'/'220929_example_signals_post_bf_fft_86deg', dpi=128)

In [ ]:
system_noise_temp = 10
n_pts_dist = 1001
n_sample = 8192
noise_variance = 1.38e-23 * 50 * 60 * 205e6 * system_noise_temp / n_sample
x = np.linspace(0, 5, n_pts_dist)

#rician_example = rician_pdf(x, noise_variance, np.array([3e-7]))

rayleigh_example = rician_pdf(x, 1, np.array([0])).squeeze()
norm = np.trapz(rayleigh_example, x)
rayleigh_example /= norm
rayleigh_cdf = cdf_from_pdf(x, rayleigh_example[np.newaxis, :]).squeeze()

rayleigh_cdf100 = rayleigh_cdf ** 100

sns.set_theme(style='whitegrid', context='talk', font_scale=1.25)
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(13, 8),)


axs.plot(x, rayleigh_cdf, label=r'$N_{Template}=1$')
axs.plot(x, rayleigh_cdf100, label=r'$N_{Template}=100$')
#axs.plot(x, rician_example.squeeze(), label='Rician (Signal Bin)')
axs.set_ylabel('Probability (AU)')
axs.set_xlabel('Matched Filter Score Threshold')
axs.legend(loc=4, title='Background CDF')

plt.savefig(Path.home()/'group'/'project'/'plots'/'220929_rayleigh_background_cdf', dpi=128)